In [8]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

df = pd.read_csv('./raw_data/VOO_60min_splits_only_standard.csv', parse_dates=['timestamp'], index_col='timestamp').sort_index()
df = df[(df.index >= "2018-01-01") & (df.index < "2025-01-01")]

volume = df['volume'].ewm(span=5).mean()

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=df.index, y=df['close'], name="Close Price"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df.index, y=volume, name="Volume", line=dict(color='orange')),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Close Price and Volume"
)

# Set x-axis title with reduced tick density
fig.update_xaxes(
    title_text="Date", 
    type="category",
    tickmode='linear',
    tick0=0,
    dtick=50  # Show every 50th tick to reduce density
)

# Set y-axes titles
fig.update_yaxes(title_text="Close Price", secondary_y=False)
fig.update_yaxes(title_text="Volume", secondary_y=True)

fig.show()